# Read in from anupam and siphon flat materials

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
device = gpus[0]
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
df = pd.read_csv('Compound_flatness_score.csv')

In [9]:
df

,Unnamed: 0,MP-ID,Comp Flatness score,Flat/not-flat,Flat segments
0,0,mp-1,0.189216,False,[]
1,1,mp-2,0.197447,False,[]
2,2,mp-3,0.350097,False,[]
3,3,mp-4,0.143728,False,[]
4,4,mp-7,0.799074,True,[4]
...,...,...,...,...,...
69814,69814,mp-1446611,0.285079,False,[]
69815,69815,mp-1475723,0.172382,False,[]
69816,69816,mp-1475733,0.922906,True,"[1, 2, 4]"
69817,69817,mp-1503976,0.130819,False,[]


In [10]:
df_slice = df[['MP-ID','Flat/not-flat']]
# extract materials with at least one flat band across the whole k-space path chosen
df_flat_only = df_slice.loc[df_slice['Flat/not-flat']==True]
df_flat_only

,MP-ID,Flat/not-flat
4,mp-7,True
18,mp-25,True
26,mp-35,True
33,mp-43,True
64,mp-83,True
...,...,...
69787,mp-1443188,True
69788,mp-1443279,True
69791,mp-1443442,True
69806,mp-1445771,True


In [11]:
flat_ids = df_flat_only['MP-ID'].to_numpy()
#print(flat_ids)
np.savetxt('flat_mat_ids', flat_ids, delimiter=',', fmt='%s')

In [13]:
!pip install ase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.8 MB/s eta 0:00:0000:0100:01


In [17]:
!pip install mp_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 1.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.5/190.5 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.2/395.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.5/676.5 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━

## Using the 2nd version of the synthesizability predictor

#### Order the .CIF files in terms of rising material project-id

In [1]:
import re
from xiespp import synthesizability, get_test_samples
# or 
#from xiespp import synthesizability_2 as synthesizability

In [2]:
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
def atoi(text):
    return int(text) if text.isdigit() else text

In [13]:
samples = get_test_samples('CSi')
samples.sort(key=natural_keys)
print(len(samples))

2


In [8]:
samples[1]

'/notebooks/XIE-SPP/xiespp/synthesizability_1/data/all_flat_mats/mp-25.cif'

In [28]:
model = synthesizability.SynthesizabilityPredictor(device=device)
model

<SynthesizabilityPredictor: device=PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), ensemble=100>

In [ ]:
scores = model.predict(samples)

Reading CIF files...


100%|██████████| 33399/33399 [08:42<00:00, 63.92it/s] 


Preparing image objects...
Checking image requirements...


100%|██████████| 33399/33399 [03:06<00:00, 179.14it/s] 


Valid images:  33,334 /  33,399
Preparing point clouds...


  0%|          | 152/33334 [05:25<25:58:52,  2.82s/it]

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Command line usage
~~~sh
(syn) ➜  ~ xiespp_synthesizability --test -v
~~~
or
~~~sh
(syn) ➜  ~ synthesizability --test -v
~~~